### 專案名稱: 第四屆機器學習百日馬拉松
### 功能描述: 第54天作業
### 版權所有: Dunk
### 程式撰寫: Dunk
### 撰寫日期：2020/05/12
### 改版日期:  
### 改版備註:

# 作業
* 試著想想看, 非監督學習是否有可能使用評價函數 (Metric) 來鑑別好壞呢?  
(Hint : 可以分為 "有目標值" 與 "無目標值" 兩個方向思考)

### ANS:

> 其實非監督評價是很困難的, 光是是否要參考目標值本身就是一大難題  
> 也因此非監督模型的評價, 有無目標值都有各自的模型可參考  
> 簡單來說就是兩者都可以, 但是無目標值的評價方式似乎更合理一些  
> 有目標值時, 評估的方式類似分類問題, 但仍須確定生成的分類與原訂的標籤對照關係  
> 無目標值的評價法, 比較接近非監督的性質本身, 同一群內資料越靠近/不同群資料越遠 是其主要考量

#### 評估 ####
- 最大困難
>與監督模型不同，非監督因為沒有目標值，因此無法使用目標值的預估與實際差距，來評估模型的優劣

- 評估方式類型
> 有目標值的
> 
>- 如果資料有目標值，只是先忽略目標值做非監督學習，則只要微調後，就可以使用原本監督的測量函數評估準確性
>
>無目標值的
>
>- 但通常沒有目標值/目標值非常少才會用非監督模型，這種情況下，只能使用資料本身的分布資訊，來做模型的評估

>基本上非監督學習的輸出結果，通常是在協助我們整理資料與了解資料，結果的優劣比較主觀，不容易用評價函數來鑑別。
>
>以分群來說我們如果有目標值的預期，似乎有一定的衡量標準，但是結果的呈現是否如我們的預期分群似乎是一個很大的問題，但是真的能給很多線索參考。"

无监督学习的评估的确是比较复杂的，也没有标准答案。一般来说，无监督学习主要是为了挖掘数据中的局部关系(locality)或者局部和全局之间的关系，往往评估方法也都是依赖于 
>- (i) 密度或距离 
>- (ii) 统计学手段 
>- (iii) 监督学习的方法 。

无监督学习中应用较广的子分类包括：

- 聚类(clustering)，如k-means/层次聚类(Hierarchical Clustering)等
- 维度压缩(dimension reduction)，如PCA等
- 异常检测(normality detection)，如iForest等
其他还有流形学习相关的模型，以及不少神经网络相关的(如自编码器)也可以被归类到如维度压缩里面。

让我们先给出一些经验建议，再从理论上进行分析。本文的目的不是给出算法介绍，需要了解的细节的朋友推荐参考文中提及的资料。

1. 将无监督变成“有监督”学习 - 生成标签
有不少研究致力于将无监督转化为监督学习，之后就可以用我们熟悉的方法，如准确率(accuracy)进行评估。

比如有研究用了顺序集成(Sequential Ensemble)的方法进行异常检测[2]，在每一次迭代的时候，通过分析多个基学习器的相关系，生成了伪标签用来淘汰表现不佳的基学习器。从头到尾，它们也并未有真正的标签，而是通过集成的方法生成了可信度较高的标签，从而转换为了一种监督手段。

2. 相似度分析 (similarity analysis)
文章一开始就提到了无监督学习往往都依赖于数据间的相似度，这可以表现为空间上的密度，也可以表现为距离度量，而二者界限不总是那么明显。

以异常检测为例，我们假设大量的正常数据都紧密相邻(可以形成多个簇)，而异常值往往和这些正常点有较大的不同。前面提到的iForest和大部分异常检测算法的核心思想都是将异常点隔离(isolate)出去，通过其所在空间的低密度实现。

从聚类角度来看，我们希望属于同一个簇的数据尽量相似，而不同簇之间的差别越大越好。一般常见的就是衡量：

簇内相似性：越相似越好
簇间差异性：越不同越好
好的聚类应该满足同时有高簇内相似性和高簇间差异性。而衡量这些指标，往往依然是依赖于距离度量或者密度度量。最常见的算法有求簇内每个点到簇中心的几何距离的均值，更加复杂的方法可以参看[3]章节9.3.

3. 分布拟合、排序、统计学方法
除了上面提到的常用方法，还有很多具体方法：

如果我们对无监督学习的任务有清晰的了解(有先验)，那么可以尝试直接对分布进行拟合。如[2]用exponential分布+高斯分布来拟合异常点和正常点。比如，假设模型输出的结果是异常分数，那么应该是少数点的异常分较高(符合exp分布)，大部分点的异常分较低(符合正态分布)。那么评估标准也可以是数据的拟合程度。
信息论和统计学方法也给出很多思路来测量相似性，或者作为“距离度量”。如Mutual information，和另一位答主提到的normalized mutual information。值得一提的是，mutual information只适合用于衡量离散随机变量，如果要做离散和连续随机变量可以使用 maximal information coefficient。

4. 不同模型之间的对比和融合
如果你训练和多个相似的模型，假设十个不同k取值的k-means，那么你会想要：

这些模型中哪个最好？
是否可以把这些模型整合起来做集成？
如果你训练了多个不同模型，假设一个k-means和一个层次聚类，你想要：

这两个模型哪个更好？
是否可以把这两个模型整合起来做集成？
首先，无监督情况下不同模型进行横向对比比较困难，一般不推荐。但集成学习往往可以帮助提高无监督学习的结果，具体的讨论可以参考[5]。一般来说，整合多个基学习器，如k-means和层次聚类前，需要对其结果进行标准化处理到均值为0标准差为1的正态分布上去，之后一般可以：

对标准化的结果做排序(ranking)，或者直接对排序结果集合。
取最大或者最小值。
取平均(Averaging)，也叫做majority vote，就是对所有基学习器结果求平均少数服从多数。
加权平均：和平均法类似，但不同的基学习器根据其性能有不同的权重，能力越强的分类器权重越大。

5. 总结
总结来看，无监督学习的衡量没有标准方法，具体的应用会导致不同的评估方法。从聚类来看，距离度量和密度估计是比较直观的判断方法。异常检测也有类似的性质，但往往比聚类更难(因为数据不平衡)。

除了距离度量，互信(mutual information)等类似的手段也可以用作衡量相似度和差异度。如果对数据和问题有更深刻的理解，还可以假设概率分布[4]，虽然这可遇不可求。某些特定情况下，我们还可以尝试生成标签，将问题转化为监督学习[2]。

正因为无监督学习面临如此复杂的场景，所以比较可靠的手段是尝试使用集成学习，更多相关的信息可以参考[6]。

#### 【火爐煉AI】機器學習024-無監督學習模型的性能評估--輪廓係數
(本文所使用的Python庫和版本號: Python 3.6, Numpy 1.14, scikit-learn 0.19, matplotlib 2.2 )

前面我們學習過監督學習模型的性能評估，由於數據集有標記，所以我們可以將模型預測值和真實的標記做比較，計算兩者之間的差異，從而來評估監督學習模型的好壞。

但是，對於無監督學習模型，由於沒有標記數據，我們該怎麼樣評估一個模型的好壞了？顯然，此時我們不能採用和監督學習模型一樣的評估方式了，而要另闢蹊徑。


1. 度量聚類模型的好壞---輪廓係數
有很多種度量聚類模型的算法，其中一個比較好用的算法就是輪廓係數（Silhouette Coefficient）指標。這個指標度量模型將數據集分類的離散程度，即判斷數據集是否分離的合理，判斷一個集群中的數據點是不是足夠緊密（即內聚度），一個集群中的點和其他集群中的點相隔是否足夠遠（即分離度），故而輪廓係數結合了內聚度和分離度這兩種因素，可以用來在相同原始數據的基礎上用來評價不同算法，或者算法不同運行方式對聚類結果所產生的影響。

以下是百度對輪廓係數的說明，此處我直接搬過來用了。

輪廓係數的計算方式說明


2. 使用輪廓係數評估K-means模型
首先是用pandas加載數據集，查看數據集加載是否正確，這部分可以看我的具體代碼，此處省略。

然後我隨機的構建一個K-means模型，用這個模型來訓練數據集，並用輪廓係數來評估該模型的優虐，代碼如下：

<code>
    
    from sklearn.cluster import KMeans
    # 构建一个聚类模型，此处用K-means算法
    model=KMeans(init='k-means++',n_clusters=3,n_init=10) 
    # 原始K-means算法最开始随机选取数据集中K个点作为聚类中心，
    # 分类结果会因为初始点的选取不同而有所区别
    # 而K-means++算法改变这种随机选取方法，能显著的改善分类结果的最终误差
    # 此处我随机的指定n_cluster=3，看看评估结果
    model.fit(dataset)    
    # 使用轮廓系数评估模型的优虐
    from sklearn.metrics import silhouette_score
    si_score=silhouette_score(dataset,model.labels_,
                          metric='euclidean',sample_size=len(dataset))
    print('si_score: {:.4f}'.format(si_score))

    -------------------------------------輸---------出-- ------------------------------

    si_score: 0.5572

    --------------------------------------------完----- --------------------------------
</code>    

從上面的代碼可以看出，計算輪廓係數是非常簡單的。

########################小**********結############## #################

1， sklearn中已經集成了輪廓係數的計算方法，我們只需要調用該函數即可，使用非常簡單。

2， 有了模型的評估指標，我們就可以對模型進行一些優化，提升模型的性能，或者用該指標來比較兩個不同模型在相同數據集上的效果，從而為我們選擇模型提供指導。

################################################## ###############

#### 參考網址
[一个无监督学习算法，如何判断其好坏呢? - 知乎](https://www.zhihu.com/question/30855292 "一个无监督学习算法，如何判断其好坏呢? - 知乎")

[【ML入門系列】（三）監督學習和無監督學習](https://www.itread01.com/articles/1493899033.html "【ML入門系列】（三）監督學習和無監督學習")

[【火爐煉AI】機器學習024-無監督學習模型的性能評估--輪廓係數](https://juejin.im/post/5b8f3bd06fb9a019d53ea30a+ "【火爐煉AI】機器學習024-無監督學習模型的性能評估--輪廓係數")